In [2]:
import sys
sys.path+= ['/Users/ericxia/Documents/Research/iRF/irf/demo', '/Users/ericxia/anaconda/lib/python36.zip', '/Users/ericxia/anaconda/lib/python3.6', '/Users/ericxia/anaconda/lib/python3.6/lib-dynload', '/Users/ericxia/anaconda/lib/python3.6/site-packages', '/Users/ericxia/Documents/Research/iRF/irf']

In [3]:
import irf
from sklearn.datasets import load_boston, load_breast_cancer
import irf.ensemble.wrf as wrf
from irf.ensemble.wrf import wrf_reg
import numpy as np

from sklearn.model_selection import train_test_split

from irf import irf_utils
from irf import irf_jupyter_utils

/Users/ericxia/anaconda/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
/Users/ericxia/anaconda/lib/python3.6/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [4]:
sys.path

['/Users/ericxia/anaconda/lib/python36.zip',
 '/Users/ericxia/anaconda/lib/python3.6',
 '/Users/ericxia/anaconda/lib/python3.6/lib-dynload',
 '',
 '/Users/ericxia/anaconda/lib/python3.6/site-packages',
 '/Users/ericxia/Documents/Research/iRF/irf',
 '/Users/ericxia/anaconda/lib/python3.6/site-packages/IPython/extensions',
 '/Users/ericxia/.ipython',
 '/Users/ericxia/Documents/Research/iRF/irf/demo',
 '/Users/ericxia/anaconda/lib/python36.zip',
 '/Users/ericxia/anaconda/lib/python3.6',
 '/Users/ericxia/anaconda/lib/python3.6/lib-dynload',
 '/Users/ericxia/anaconda/lib/python3.6/site-packages',
 '/Users/ericxia/Documents/Research/iRF/irf']

In [5]:
from pyspark.ml.fpm import FPGrowth
from pyspark.sql import SparkSession
from pyspark.sql.types import *

In [6]:
data = load_breast_cancer()
X = data['data']
y = data['target']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(
        data.data, data.target, train_size=0.9)

In [8]:
rf = wrf()
weight = np.ones((X.shape[1],)) / X.shape[1]
rf.fit(X, y, feature_weight=weight)
print(rf.feature_importances_)

[0.00000000e+00 1.18476689e-04 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 1.75509412e-01
 0.00000000e+00 0.00000000e+00 2.70753104e-03 0.00000000e+00
 6.00027836e-03 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 1.32087922e-01 4.79954935e-02 3.18276352e-01 1.19620759e-02
 0.00000000e+00 0.00000000e+00 8.40359210e-03 2.96938866e-01
 0.00000000e+00 0.00000000e+00]


In [9]:
all_rf_tree_data = irf_utils.get_rf_tree_data(rf=rf, X_train=X_train, X_test=X_test, y_test=y_test)

In [10]:
def convert(leaf_node_paths, feature_paths):
    leafnode_to_features = {}
    for i in range(len(leaf_node_paths)):
        leaf_node = leaf_node_paths[i][-1]
        leafnode_to_features[leaf_node] = feature_paths[i]
    return leafnode_to_features

In [11]:
interactions = np.array([])
inp = X_train[y_train==1]

for i in range(len(rf.estimators_)):
    key = "dtree" + str(i)
    tree_dict = all_rf_tree_data[key]
    #assume that ith entry of feature_paths corresponds to ith entry of leaf_node_paths
    feature_paths = tree_dict["all_uniq_leaf_paths_features"]
    leaf_nodes = tree_dict["all_leaf_nodes"]
    leaf_node_paths = tree_dict["all_leaf_node_paths"]
    decision_tree = rf.estimators_[i]
    
    end = decision_tree.apply(inp)
    leafnode_to_features = convert(leaf_node_paths, feature_paths)
    temp_features = np.vectorize(leafnode_to_features.get, otypes=[np.ndarray])(end)
    interactions = np.concatenate((interactions, temp_features))
    


In [12]:
#schema = StructType([StructField("id", IntegerType(), True), 
#                     StructField("list", ArrayType(), True)])

spark = SparkSession \
    .builder \
    .appName("iterative Random Forests with FP-Growth") \
    .getOrCreate()
    
input_list = [(i, interactions[i].tolist()) for i in range(len(interactions))]

df = spark.createDataFrame(input_list, ["id", "items"])
df.show()

+---+--------------------+
| id|               items|
+---+--------------------+
|  0| [7, 21, 22, 23, 27]|
|  1| [7, 21, 22, 23, 27]|
|  2|[12, 20, 22, 23, 27]|
|  3| [7, 21, 22, 23, 27]|
|  4|[20, 21, 22, 23, 27]|
|  5| [7, 21, 22, 23, 27]|
|  6| [7, 21, 22, 23, 27]|
|  7| [7, 21, 22, 23, 27]|
|  8| [7, 21, 22, 23, 27]|
|  9|[20, 21, 22, 23, 27]|
| 10|[20, 21, 22, 23, 27]|
| 11|[20, 21, 22, 23, 27]|
| 12| [7, 21, 22, 23, 27]|
| 13| [7, 21, 22, 23, 27]|
| 14| [7, 21, 22, 23, 27]|
| 15| [7, 21, 22, 23, 27]|
| 16| [7, 21, 22, 23, 27]|
| 17|[7, 20, 21, 22, 2...|
| 18| [7, 21, 22, 23, 27]|
| 19| [7, 21, 22, 23, 27]|
+---+--------------------+
only showing top 20 rows



In [13]:
fpGrowth = FPGrowth(itemsCol="items", minSupport=0.1, minConfidence=0.6)
model = fpGrowth.fit(df)

model.freqItemsets.show(40, False)

+----------------+----+
|items           |freq|
+----------------+----+
|[7]             |1543|
|[7, 21]         |1144|
|[7, 21, 22]     |840 |
|[7, 21, 22, 27] |534 |
|[7, 21, 27]     |579 |
|[7, 22]         |933 |
|[7, 22, 27]     |605 |
|[7, 20]         |945 |
|[7, 20, 21]     |579 |
|[7, 20, 22]     |351 |
|[7, 20, 22, 27] |320 |
|[7, 20, 27]     |658 |
|[7, 27]         |943 |
|[27]            |2580|
|[23]            |912 |
|[23, 22]        |679 |
|[23, 22, 27]    |671 |
|[23, 20]        |408 |
|[23, 20, 27]    |408 |
|[23, 27]        |904 |
|[22]            |2283|
|[22, 27]        |1955|
|[21]            |1912|
|[21, 22]        |1567|
|[21, 22, 27]    |1261|
|[21, 27]        |1347|
|[10]            |532 |
|[10, 7]         |532 |
|[10, 7, 20]     |532 |
|[10, 20]        |532 |
|[12]            |330 |
|[12, 21]        |325 |
|[12, 21, 22]    |325 |
|[12, 22]        |330 |
|[20]            |1808|
|[20, 21]        |814 |
|[20, 21, 22]    |476 |
|[20, 21, 22, 27]|453 |
|[20, 21, 27]   

In [14]:
inter = [interactions[i].tolist() for i in range(len(interactions))]

In [14]:
#If we were used to do GraphLab
import turicreate as tc
ids = tc.SArray(list(range(len(inter))))
vals = tc.SArray(inter)
sf = tc.SFrame({'id':ids,'val':vals})
model = tc.pattern_mining.create(sf, 'val',
                features=['val'], min_length = 5)

AttributeError: module 'turicreate' has no attribute 'pattern_mining'

In [15]:
from prefixspan import PrefixSpan
ps = PrefixSpan(inter)
ps.topk(100)

[(2580, [27]),
 (2283, [22]),
 (1955, [22, 27]),
 (1912, [21]),
 (1808, [20]),
 (1567, [21, 22]),
 (1543, [7]),
 (1521, [20, 27]),
 (1347, [21, 27]),
 (1261, [21, 22, 27]),
 (1144, [7, 21]),
 (945, [7, 20]),
 (943, [7, 27]),
 (933, [7, 22]),
 (932, [20, 22]),
 (912, [23]),
 (904, [23, 27]),
 (901, [20, 22, 27]),
 (840, [7, 21, 22]),
 (814, [20, 21]),
 (679, [22, 23]),
 (671, [22, 23, 27]),
 (658, [7, 20, 27]),
 (605, [7, 22, 27]),
 (579, [7, 20, 21]),
 (579, [7, 21, 27]),
 (539, [20, 21, 27]),
 (534, [7, 21, 22, 27]),
 (532, [7, 10]),
 (532, [7, 10, 20]),
 (532, [10]),
 (532, [10, 20]),
 (476, [20, 21, 22]),
 (453, [20, 21, 22, 27]),
 (408, [20, 23]),
 (408, [20, 23, 27]),
 (351, [7, 20, 22]),
 (330, [12]),
 (330, [12, 22]),
 (325, [12, 21]),
 (325, [12, 21, 22]),
 (320, [7, 20, 22, 27]),
 (313, [12, 22, 27]),
 (313, [12, 27]),
 (308, [12, 21, 22, 27]),
 (308, [12, 21, 27]),
 (307, [21, 22, 23]),
 (307, [21, 23]),
 (305, [21, 22, 23, 27]),
 (305, [21, 23, 27]),
 (304, [7, 20, 21, 27]),

In [ ]:
####WILL KILL THE KERNEL FOR SOME REASON
stability_score = irf_utils.run_iRF(X_train=X_train,
                                    X_test=X_test,
                                    y_train=y_train,
                                    y_test=y_test,
                                    rf = rf)

In [16]:
import pyfpgrowth

In [19]:
fp_tree = pyfpgrowth.FPTree(inter, 0.9, None, None)

In [33]:
children = fp_tree.root.children

In [37]:
children[0].value

27